In [444]:
import requests
import random
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime , timedelta
import re
import os



path = "//Users//hajiaga//Desktop//DataScience//chromedriver"

driver = webdriver.Chrome()

driver.implicitly_wait(5)


In [445]:
def azetoengdate(string):

    gun = string[:2]

    ay = str(re.findall(r'\D+', string)).replace(" ","").lower()

    if ay == "yanvar":
        numay = "01"
    elif ay == "fevral":
        numay = "02"
    elif ay == "mart":
        numay = "03"
    elif ay == "aprel":
        numay = "04"
    elif ay == "may":
        numay = "05"
    elif ay == "iyun":
        numay = "06"
    elif ay == "iyul":
        numay = "07"
    elif ay == "avqust":
        numay = "08"
    elif ay == "sentyabr":
        numay = "09"
    elif ay == "oktyabr":
        numay = "10"
    elif ay == "noyabr":
        numay = "11"
    elif ay == "dekabr":
        numay = "12"
    else:
        numay ="Nan"
    
    return f"{numay} {gun}"

In [446]:
def scrape_new_window(href , dic):
    time.sleep(1)
    driver.get(href)

    dic["full_name"].append((driver.find_element(By.XPATH , "//h1[@class = 'product-title']")).text)

    temp = driver.find_element(By.XPATH , "//div[@class = 'product-properties__column']")

    dic["location"].append((temp.find_element(By.XPATH , "//span[@class = 'product-properties__i-value']")).text)

    dic["comp_name"].append((temp.find_element(By.TAG_NAME , "a")).text)

    tempp = driver.find_element(By.XPATH , "//div[@class = 'product-price__i product-price__i--bold']")
    
    temppspans = tempp.find_elements(By.TAG_NAME , "span")

    dic["price"].append(temppspans[0].text)

    dic["currency"].append(temppspans[1].text)

    temp1 = driver.find_element(By.XPATH , "//div[@class = 'product-info product-info__statistics']")

    temp2 = temp1.find_elements(By.TAG_NAME , "div")

    date = (str(temp2[1].find_element(By.TAG_NAME , "span").text)).split(',')[0]

    if date == "Bugün":
        today = datetime.today()

        dic["date"].append(today.strftime('%m %d'))
        
    elif date == "Dünən":

        yesterday = datetime.today() - timedelta(days=1)

        dic["date"].append(yesterday.strftime('%B %d'))
    else:
        dic["date"].append(azetoengdate(date))

    driver.back()

    return dic


In [447]:
path1 = "https:/tap.az"

driver.get(path1)

time.sleep(5)
try:
    category = driver.find_element(By.XPATH , "//a[@data-for = 'consumer-electronics']")

    driver.get(category.get_attribute('href'))

    dropdown = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'price']")

    dropdown.click()
    
    minprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_lower']")

    maxprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_upper']")

    minprice.send_keys("1350")

    maxprice.send_keys("2500")

    time.sleep(5)

    categories = driver.find_element(By.XPATH , "//div[@class = 'subcategories-inner']")

    uls = categories.find_elements(By.TAG_NAME , 'ul')

    for ul in uls:
        
        lis = ul.find_elements(By.TAG_NAME , 'li')

        for li in lis:

            if "noutbuk" in str(li.find_element(By.TAG_NAME , 'a').get_attribute('href')).lower():

                href = li.find_element(By.TAG_NAME , 'a').get_attribute('href')

                break
    
    driver.get(href)

    count_= str(href).count("/")+1

    print(count_)

    dropdown_new = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'new']")

    dropdown_new.click()

    dropdown_new.find_element(By.TAG_NAME,"ul").find_elements(By.TAG_NAME , 'li')[1].click()
    
    # time.sleep(10)

    # product_columns = driver.find_element(By.XPATH ,"//div[@class = 'js-endless-container products endless-products']" )

    time.sleep(3)

    # product = driver.find_element(By.CSS_SELECTOR , "#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div:nth-child(1)")


    dic_comp = {}

    dic_comp["location"] = []

    dic_comp["date"] = []

    dic_comp["link"] = []

    dic_comp["price"] = []

    dic_comp["comp_name"] = []

    dic_comp["full_name"] = []

    dic_comp["currency"] = []

    last_length = 0

    
    while True:

        products = driver.find_elements(By.CSS_SELECTOR, f"#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div")

        if len(products) == last_length:
            break
       
        i = 0

        for product in products:
            
            i+=1

            if i>last_length:
                
                if ("pagination" not in product.get_attribute("class")):
                    time.sleep(5)
                    
                    a = product.find_element(By.TAG_NAME , "a")

                    if (str(a.get_attribute("href")).count('/') == 6):

                        dic_comp["link"].append(a.get_attribute("href"))

                        dic_comp = scrape_new_window(a.get_attribute("href") , dic_comp)
    
        last_length = len(products)

        time.sleep(5)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(5)

    
    
    # while driver.find_element(By.CSS_SELECTOR, f"#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div:nth-child({i})"):
    #     if i%48 == 0:
    #         time.sleep(3)

    #         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #     if i >46:

    #         time.sleep(3)

    #         if ('vip' not in product.get_attribute('class').lower()):
                
    #             a = product.find_element(By.XPATH , "//a[@class = 'products-link' and @data-stat = 'ad-card-link']")

    #             dic_comp["link"].append(a.get_attribute("href"))

    #             dic_comp = scrape_new_window(a.get_attribute("href") , dic_comp)

                
    # product = driver.find_elements(By.CSS_SELECTOR, f"#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div")
            

    # print(len(product))
    # print(product)
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # time.sleep(10)
    # product = driver.find_elements(By.CSS_SELECTOR, f"#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div")
     
    # print(len(product))
    # print(product)   

    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # time.sleep(10)
    # product = driver.find_elements(By.CSS_SELECTOR, f"#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div")
     
    # print(len(product))
    # print(product)   

    # while True:

    #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #     time.sleep(1)

    #     new_height = driver.execute_script("return document.body.scrollHeight")

    #     if new_height == last_height:
    #         break

    #     last_height = new_height

    #     elements = driver.find_elements(By.CSS_SELECTOR , "#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div")

    #     for product in elements:
            
    #         a = product.find_element(By.XPATH , "//a[@class = 'products-link' and @data-stat = 'ad-card-link']")
    #         if len(dic_comp["link"]) > 0:

    #             if dic_comp["link"][-1] != a.get_attribute("href"):

    #                 dic_comp["link"].append(a.get_attribute("href"))

    #                 dic_comp = scrape_new_window(a.get_attribute("href") , dic_comp)

    #         else:
    #             dic_comp["link"].append(a.get_attribute("href"))

    #             dic_comp = scrape_new_window(a.get_attribute("href") , dic_comp)











    # The final problem is that i count until 48 but after that website is getting refreshed so i need to make a function for that and stop it gettinf the same thing after 48
            

    #Ilk usula don her defe scroll ele yeniden tap butun seyleri biraz vaxt alacaq
        


    
        
finally:
    driver.close()



6


In [449]:
print(len(dic_comp["link"]))
print(dic_comp["link"])

277
['https://tap.az/elanlar/elektronika/noutbuklar/41077250', 'https://tap.az/elanlar/elektronika/noutbuklar/41025283', 'https://tap.az/elanlar/elektronika/noutbuklar/40021517', 'https://tap.az/elanlar/elektronika/noutbuklar/41045807', 'https://tap.az/elanlar/elektronika/noutbuklar/41076901', 'https://tap.az/elanlar/elektronika/noutbuklar/41076601', 'https://tap.az/elanlar/elektronika/noutbuklar/41076441', 'https://tap.az/elanlar/elektronika/noutbuklar/41059516', 'https://tap.az/elanlar/elektronika/noutbuklar/40962615', 'https://tap.az/elanlar/elektronika/noutbuklar/41075916', 'https://tap.az/elanlar/elektronika/noutbuklar/40982459', 'https://tap.az/elanlar/elektronika/noutbuklar/41074918', 'https://tap.az/elanlar/elektronika/noutbuklar/40953802', 'https://tap.az/elanlar/elektronika/noutbuklar/41073863', 'https://tap.az/elanlar/elektronika/noutbuklar/40855345', 'https://tap.az/elanlar/elektronika/noutbuklar/41073594', 'https://tap.az/elanlar/elektronika/noutbuklar/40185879', 'https://

In [450]:
for i in dic_comp.keys():
    print(len(dic_comp[i]))
dftapaz = pd.DataFrame.from_dict(dic_comp)

filename = "Users/hajiaga/Desktop/DataScience/tapazdata.csv"
filename2 = "Users/hajiaga/Desktop/DataScience/tapazdata.xlsx"
if os.path.exists(filename):
    os.remove(filename)
if os.path.exists(filename2):
    os.remove(filename2)
dftapaz.to_csv(filename, index=False) 
dftapaz.to_excel(filename2, index=False) 

dftapaz

277
277
277
277
277
277
277


,location,date,link,price,comp_name,full_name,currency
0,Bakı,04 01,https://tap.az/elanlar/elektronika/noutbuklar/...,1 480,Apple Macbook,Apple Macbook Pro 16,AZN
1,Bakı,04 01,https://tap.az/elanlar/elektronika/noutbuklar/...,1 649,Apple Macbook,Apple Macbook Air M2,AZN
2,Bakı,04 01,https://tap.az/elanlar/elektronika/noutbuklar/...,1 500,Dell,"Noutbuk ""Dell G15 5511""",AZN
3,Bakı,04 01,https://tap.az/elanlar/elektronika/noutbuklar/...,1 350,HP,HP Victus 15-Fa1093dx,AZN
4,Bakı,04 01,https://tap.az/elanlar/elektronika/noutbuklar/...,1 570,Apple Macbook,Apple Macbook Air M1 16/256 GB,AZN
...,...,...,...,...,...,...,...
272,Bakı,04 01,https://tap.az/elanlar/elektronika/noutbuklar/...,1 650,MSI,"Noutbuk ""MSI""",AZN
273,Bakı,March 31,https://tap.az/elanlar/elektronika/noutbuklar/...,1 400,Asus,Asus TUF Gaming,AZN
274,Bakı,Nan 30,https://tap.az/elanlar/elektronika/noutbuklar/...,1 350,Dell,"Noutbuk ""Dell XPS 15 7590""",AZN
275,Bakı,March 31,https://tap.az/elanlar/elektronika/noutbuklar/...,1 350,MSI,MSİ GF63 Thin,AZN
